<a href="https://colab.research.google.com/github/redatabich/BIGDATALABS/blob/main/Lab_Spark_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark


In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("FIFA World Cup Analysis") \
    .getOrCreate()

print("Spark version :", spark.version)


Spark version : 4.0.1


In [3]:
df = spark.read.csv("/content/fifaworldcup.csv", header=True, inferSchema=True)

df.printSchema()
df.show(5)


root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: string (nullable = true)
 |-- away_score: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)

+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|      date|home_team|away_team|home_score|away_score|tournament|   city| country|neutral|
+----------+---------+---------+----------+----------+----------+-------+--------+-------+
|1872-11-30| Scotland|  England|         0|         0|  Friendly|Glasgow|Scotland|  false|
|1873-03-08|  England| Scotland|         4|         2|  Friendly| London| England|  false|
|1874-03-07| Scotland|  England|         2|         1|  Friendly|Glasgow|Scotland|  false|
|1875-03-06|  England| Scotland|         2|         2|  Friendly| London| England|  false|


In [4]:
df.createOrReplaceTempView("matches")


In [5]:
spark.sql("""
SELECT COUNT(*) AS total_matchs
FROM matches
""").show()


+------------+
|total_matchs|
+------------+
|       44152|
+------------+



In [7]:
spark.sql("""
SELECT tournament, COUNT(*) AS nb_matchs
FROM matches
GROUP BY tournament
ORDER BY nb_matchs DESC
LIMIT 10
""").show()


+--------------------+---------+
|          tournament|nb_matchs|
+--------------------+---------+
|            Friendly|    17461|
|FIFA World Cup qu...|     7774|
|UEFA Euro qualifi...|     2593|
|African Cup of Na...|     1932|
|      FIFA World Cup|      948|
|        Copa América|      841|
|AFC Asian Cup qua...|      764|
|African Cup of Na...|      742|
|          CECAFA Cup|      620|
|CFU Caribbean Cup...|      606|
+--------------------+---------+



In [10]:
spark.sql("""
SELECT COUNT(*) AS matchs_neutres
FROM matches
WHERE neutral = true
""").show()



+--------------+
|matchs_neutres|
+--------------+
|         10996|
+--------------+



In [11]:
spark.sql("""
SELECT country, COUNT(*) AS nb_matchs
FROM matches
GROUP BY country
ORDER BY nb_matchs DESC
LIMIT 10
""").show()


+--------------------+---------+
|             country|nb_matchs|
+--------------------+---------+
|       United States|     1259|
|              France|      830|
|            Malaysia|      752|
|             England|      722|
|              Sweden|      659|
|               Qatar|      618|
|             Germany|      608|
|              Brazil|      569|
|               Spain|      568|
|United Arab Emirates|      548|
+--------------------+---------+



In [12]:
spark.sql("""
SELECT COUNT(*) AS matchs_nuls
FROM matches
WHERE home_score = away_score
""").show()


+-----------+
|matchs_nuls|
+-----------+
|      10213|
+-----------+



In [14]:
spark.sql("""
SELECT team, COUNT(*) AS nb_matchs
FROM (
    SELECT home_team AS team FROM matches
    UNION ALL
    SELECT away_team AS team FROM matches
)
GROUP BY team
ORDER BY nb_matchs DESC
""").show(10)


+-----------+---------+
|       team|nb_matchs|
+-----------+---------+
|     Sweden|     1052|
|    England|     1047|
|     Brazil|     1019|
|  Argentina|     1014|
|    Germany|      986|
|    Hungary|      964|
|     Mexico|      926|
|    Uruguay|      919|
|South Korea|      904|
|     France|      875|
+-----------+---------+
only showing top 10 rows


In [19]:
df.printSchema()



root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: string (nullable = true)
 |-- away_score: string (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)



In [20]:
from pyspark.sql.functions import year, col

df_clean = df \
    .withColumn("year", year("date")) \
    .withColumn("home_score", col("home_score").cast("int")) \
    .withColumn("away_score", col("away_score").cast("int"))

df_clean.printSchema()
df_clean.createOrReplaceTempView("matches")

root
 |-- date: date (nullable = true)
 |-- home_team: string (nullable = true)
 |-- away_team: string (nullable = true)
 |-- home_score: integer (nullable = true)
 |-- away_score: integer (nullable = true)
 |-- tournament: string (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- neutral: boolean (nullable = true)
 |-- year: integer (nullable = true)



In [23]:
spark.sql("""
SELECT tournament, year, COUNT(*) AS nb_matchs
FROM matches
GROUP BY tournament, year
ORDER BY tournament, year
""").show(10)




+---------------+----+---------+
|     tournament|year|nb_matchs|
+---------------+----+---------+
|ABCS Tournament|2010|        4|
|ABCS Tournament|2011|        4|
|ABCS Tournament|2012|        4|
|ABCS Tournament|2013|        4|
|ABCS Tournament|2015|        4|
|  AFC Asian Cup|1956|        6|
|  AFC Asian Cup|1960|        6|
|  AFC Asian Cup|1964|        6|
|  AFC Asian Cup|1968|       10|
|  AFC Asian Cup|1972|       13|
+---------------+----+---------+
only showing top 10 rows
